This example it is going to be used model 'models/b2.p'. It will be taken the dataset data with 'not_commenter'=0.0, pre-process, and predict how many Positives and Negatives are there

In [1]:
import numpy as np
import csv
import pickle
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegressionCV
%matplotlib inline

Load Data

In [2]:
file_name = "dataset.csv"
df = pd.read_csv("db/survey-no-response/v1/"+file_name)

Pick the 'not_commenter' = 0

In [3]:
df = df.loc[df['not_commenter']==0]
df.reset_index(drop=True, inplace=True)
df = df.drop(columns=['not_commenter'])

Load max and min from training

In [4]:
bowmax, bowmin = pickle.load(open( "results/bow_mm.p", "rb" ))
dfmax, dfmin = pickle.load(open( "results/df_mm.p", "rb" ))

PreProcess

In [5]:
campaigns = []
for name in df.columns:
    if "campaign." in name:
        campaigns.append(name)

campaigns_df = df[campaigns]
df = df.drop(columns=campaigns)

words = []
for w in df.columns:
    if "text." in w:
        words.append(w)

bow_df = df[words]
df = df.drop(columns=words)

city = []
for c in df.columns:
    if "city_state." in c:
        city.append(c)

city_df = df[city]
df = df.drop(columns=city)

In [6]:
normalized_bow= (bow_df-bowmin)/(bowmax-bowmin)
normalized_df = (df-dfmin)/(dfmax-dfmin)

# All the subsets with no PCA

baseline_df = pd.concat([normalized_df[['date_disseminated','date_received','submitted','email_confirmation']]],axis=1)

baseline_cam_df = pd.concat([baseline_df,campaigns_df],axis=1)

baseline_cam_bins_df = pd.concat([baseline_cam_df,normalized_df[['campaign_centered_bin','campaign_submitted_bin']]],axis=1)

baseline_bow_df = pd.concat([baseline_df,normalized_bow],axis=1)

baseline_bow_bins_df = pd.concat([baseline_bow_df,normalized_df[['all_centered_bin','all_submitted_bin']]],axis=1)

Load Model

In [7]:
model = pickle.load(open( "models/b2.p", "rb" ))
predictions = model.predict(baseline_cam_df)

Predict

In [8]:
sum(predictions == 1)

181007

In [9]:
sum(predictions == -1)

255820

In [10]:
predictions.shape[0]

436827